# AIFriend

Inteligencia artificial para hablar

# Temas a tratar
- Procesamiento del lenguaje natural.
- Conversión de voz a texto con Whisper.
- Uso de modelos de terceros.
- Conversión de texto a voz con CoquiTTS y XTTS2


# Problema

En una generación cada vez más sumergida en la tecnología, las personas cada vez hablan menos hasta el punto que se vuelven incompetentes socialmente. Esto sobre todo, a la falta de personas con las qué hablar.


# Solución

Una IA para conversar como si fuese un amigo

# Objetivo

Desarrollar una aplicación para conversar escucha y habla con la IA.

# Desarrollo

## Análisis y preprocesamiento de datos

Para este caso vamos a usar modelos preentrenados así que no hay mucho que tengamos que hacer en preprocesamiento, por ahora solo creemos un helper para nuestras pruebas

In [7]:
import numpy as np
def preprocess(audio):
  if audio.frame_rate != 16000: # 16 kHz
      audio = audio.set_frame_rate(16000)
  if audio.sample_width != 2:   # int16
      audio = audio.set_sample_width(2)
  if audio.channels != 1:       # mono
      audio = audio.set_channels(1)
  arr = np.array(audio.get_array_of_samples())
  arr = arr.astype(np.float32)/32768.0
  return arr

## Selección e implementación de modelos

Para convertir voz a texto usaremos whisper, para obtener respuesta usaremos el modelo llama que desplegamos la semana pasada, y para texto a voz usaremos CoquiTTS.

El primer paso será grabar algo para verificar que whisper funciona y que de paso nos sirva para clonar nuestra voz eventualmente con XTTS2.

In [1]:
# @title Helper para grabar nuestra voz
from IPython.display import Javascript
from google.colab import output
from base64 import b64decode
from io import BytesIO
!pip -q install pydub
from pydub import AudioSegment

RECORD = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise((resolve, reject) => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.target.result)
  reader.onerror = e => reject(new Error("Failed to read blob"))
  reader.readAsDataURL(blob)
})
var recordUntilSilence = time => new Promise(async (resolve, reject) => {
  let stream, recorder, chunks, blob, text, audioContext, analyser, dataArr, silenceStart, threshold = 50, silenceDelay = 2000
  try {
    stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  } catch (err) {
    return reject(new Error("Failed to get media stream"))
  }
  audioContext = new AudioContext()
  const source = audioContext.createMediaStreamSource(stream)
  analyser = audioContext.createAnalyser()
  analyser.fftSize = 512
  dataArr = new Uint8Array(analyser.frequencyBinCount)
  source.connect(analyser)
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.onstop = async () => {
    blob = new Blob(chunks)
    try {
      text = await b2text(blob)
      resolve(text)
    } catch (err) {
      reject(new Error("Failed to convert blob to text"))
    }
  }
  recorder.onerror = e => reject(new Error("Recorder error"))
  recorder.start()
  const checkSilence = () => {
    analyser.getByteFrequencyData(dataArr)
    const avg = dataArr.reduce((p, c) => p + c, 0) / dataArr.length

    if (avg < threshold) {
      if (silenceStart === null) silenceStart = new Date().getTime()
      else if (new Date().getTime() - silenceStart > silenceDelay) {
        recorder.stop()
        audioContext.close()
        return
      }
    } else {
      silenceStart = null
    }
    requestAnimationFrame(checkSilence)
  }
  silenceStart = null
  checkSilence()
})
console.log("JavaScript code executed successfully.")
"""

def record_until_silence():
  try:
    display(Javascript(RECORD))
    s = output.eval_js('recordUntilSilence()')
    b = b64decode(s.split(',')[1])
    audio = AudioSegment.from_file(BytesIO(b))
    return audio
  except Exception as e:
    print(f"An error occurred: {e}")
    return None

Grabamos nuestra voz y lo reproducimos

In [2]:
audio = record_until_silence()
audio

<IPython.core.display.Javascript object>

In [56]:
# Guardamos el archivo por si acaso lo necesitamos
audio.export("output.wav", format="wav")


<_io.BufferedRandom name='output.wav'>

Ahora vamos a instalar Whisper descargado directamente de GitHub (recuerda que se puede usar con la transformers API de Hugging Face o via API)

In [4]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [5]:
!pip install git+https://github.com/openai/whisper.git

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-48jm1d86
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-48jm1d86
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-m

Cargamos el modelo base (hay cientos de modelos preentrenados de whisper, los básicos son el small, el base y el huge), pero en hugging face se encuentran más de 600.

In [8]:
import whisper
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 150MiB/s]


Transcribimos nuestro mensaje que grabamos

In [ ]:
result = model.transcribe(preprocess(audio), language='es')

Vemos el resultado

In [9]:
result['text']

''

Misma solución pero con Hugging Face

In [10]:
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", "openai/whisper-base")

pipe(preprocess(audio))

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/290M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.81k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.


{'text': ' 1 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2 %, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2%, 2 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min 10 min'}

Ahora vamos a implementar nuestra solución de texto a voz, para ello usaremos CoquiTTS, vamos a instalarlo

In [11]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 938.0/938.0 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 68.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 41.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 95.5 MB/s eta 0:00:00
  Preparing metadata (setup.py

Vamos a inicializar el modelo XTTS2. Este modelo entrenado por Coqui es uno de los más rápidos en inferencia y más precisos a la hora de general lenguaje. Además, son capaces de clonar la voz con un audio de menos de 5 segundos.

In [12]:
import torch
from TTS.api import TTS

# Obtenemos el dispositivo
device = "cuda" if torch.cuda.is_available() else "cpu"

# Lista de los modelos TTS 🐸
print(TTS().list_models())

# Inicializamos el modelo XTTS2
tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > You must confirm the following:
 | > "I have purchased a commercial license from Coqui: licensing@coqui.ai"
 | > "Otherwise, I agree to the terms of the non-commercial CPML: https://coqui.ai/cpml" - [y/n]
 | | > y
 > Downloading model to /root/.local/share/tts/tts_models--multilingual--multi-dataset--xtts_v2


100%|█████████▉| 1.86G/1.87G [00:43<00:00, 61.5MiB/s]
100%|██████████| 1.87G/1.87G [00:44<00:00, 41.8MiB/s]
100%|██████████| 4.37k/4.37k [00:00<00:00, 7.52kiB/s]

100%|██████████| 361k/361k [00:00<00:00, 452kiB/s]
100%|██████████| 32.0/32.0 [00:00<00:00, 34.6iB/s]
 85%|████████▍ | 6.55M/7.75M [00:00<00:00, 18.6MiB/s]

 > Model's license - CPML
 > Check https://coqui.ai/cpml.txt for more info.
 > Using model: xtts


100%|██████████| 7.75M/7.75M [00:14<00:00, 18.6MiB/s]

Ya que cargamos el modelo, generemos una voz para el siguiente texto, clonando la voz que usamos al principio

In [13]:
tts.tts_to_file(text="Un transformador (en inglés: transformer) es un modelo de aprendizaje profundo que usa el mecanismo de autoatención, en el que se le da un peso diferente a cada parte del input.", speaker_wav="output.wav", language="es", file_path="output_tts.wav")

 > Text splitted to sentences.
['Un transformador (en inglés: transformer) es un modelo de aprendizaje profundo que usa el mecanismo de autoatención, en el que se le da un peso diferente a cada parte del input.']
 > Processing time: 38.44534373283386
 > Real-time factor: 1.332524064581164


'output_tts.wav'

Perfecto, vamos a escucharla

In [14]:
from IPython.display import Audio
Audio('output_tts.wav')

Como puedes ver, sigue siendo algo robótica la voz, y pese a que sí hay medio indicios de la voz clonada, no es perfecta. Para conseguir mejores resultados se recomienda entrenar el modelo en la voz elegida en vez de usar este mecanismo por ahora...

## Despliegue

Finalmente, usemos el modelo que desplegamos la semana pasada. Se podía usar también alguno de transformers de Hugging face o crear uno desde una librería como llama cpp, pero como cada modelo necesita mucha RAM, y ya tenemos dos modelos cargados, es preferible tener este modelo en otro lado.

In [31]:
import requests

url = 'https://text-generation.roandrad.workers.dev/'

def generate_response(chat_history):
  print(chat_history)
  response = requests.post(url, json={
      "messages": chat_history
  })
  r = response.json()
  return r['response']

Creamos una función que reciba los bytes de audio para transcribir lo que dijo el usuario

In [16]:
import whisper
model = whisper.load_model("base")
def listen_to_audio(audio_stream):
  with open('input.wav', 'wb') as f:
    f.write(audio_stream)
  result = model.transcribe('input.wav', language='es')
  return result['text']

Cargamos XTTS2

In [17]:
import torch
from TTS.api import TTS

# Get device
device = "cuda" if torch.cuda.is_available() else "cpu"

tts = TTS("tts_models/multilingual/multi-dataset/xtts_v2").to(device)

 > tts_models/multilingual/multi-dataset/xtts_v2 is already downloaded.
 > Using model: xtts


Creamos una función para generar una voz hablada a partir del texto

In [18]:
def generate_speech(text, speaker="rolando.wav"):
  speech = tts.tts_to_file(text=text, speaker_wav=speaker, language="es", file_path="output.wav")
  with open("output.wav", "rb") as f:
    return f.read()

Y otra para devolver el audio codificado en base64

In [19]:
import base64

def stream_to_base64(stream):
  """Converts a stream to a base64 data URL."""
  b64_data = base64.b64encode(stream)
  return f"data:audio/wav;base64,{b64_data.decode('utf-8')}"


Finalmente vamos a crear una función final que dada la entrada de un usuario en audio, y su historial de mensajes, transcriba el audio, genere una respuesta a ese mensaje, y convierta esa respuesta a voz, retornando todo en un json.

In [21]:
def voice_chat(audio, messages):
  text_input = listen_to_audio(audio)
  messages.append({"role": "user", "content": text_input})
  print(messages)
  response = generate_response(messages)
  speech = generate_speech(response)
  b64_data = stream_to_base64(speech)
  return {
      "input": text_input,
      "text": response,
      "audio": b64_data
  }

In [ ]:
!pip install --ignore-installed Flask==3.0.0 pyngrok==7.1.2
ngrok_key = "Coloca tu clave de ngrok aquí"
port = 5000

from pyngrok import ngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.7/99.7 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.3/227.3 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.3/133.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 705.5/705.5 kB 12.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires numpy<2.0.0,>=1.23.5, but you have numpy 1.22.0 which is incompatible.


In [52]:
ngrok.set_auth_token(ngrok_key)
ngrok.connect(port)

Exception in thread Thread-51 (_monitor_process):
Traceback (most recent call last):
  File "/usr/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.10/threading.py", line 953, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.10/dist-packages/pyngrok/process.py", line 140, in _monitor_process
    self._log_line(self.proc.stdout.readline())
  File "/usr/lib/python3.10/encodings/ascii.py", line 26, in decode
    return codecs.ascii_decode(input, self.errors)[0]
UnicodeDecodeError: 'ascii' codec can't decode byte 0xc2 in position 183: ordinal not in range(128)


<NgrokTunnel: "https://617b-35-202-136-17.ngrok-free.app" -> "http://localhost:5000">

In [54]:
from IPython.display import IFrame
IFrame(src="https://projects.rolandoandrade.me/aifriend/", width=800, height=600)

In [53]:
from flask import Flask, request, jsonify, Response
import json

app = Flask(__name__)

@app.route("/", methods=['POST'])
def hello():
  # application/multipart with audio (blob) and messages `array`
  audio_blob = request.files.get("audio")
  audio_bytes = audio_blob.read()
  messages = json.loads(request.form.get('messages'))
  chat = voice_chat(audio_bytes, messages)
  return Response(json.dumps(chat), headers=[('Access-Control-Allow-Origin', '*')])

if __name__ == '__main__':
    app.run(port = port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


[{'role': 'user', 'content': ' Hola'}]
[{'role': 'user', 'content': ' Hola'}]
 > Text splitted to sentences.
['Hola!', '¡Bienvenido!', 'How can I assist you today?', "Do you have a specific question or topic you'd like to discuss?", "I'm here to help with any information you might need."]


INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 17:58:09] "POST / HTTP/1.1" 200 -


 > Processing time: 9.031518936157227
 > Real-time factor: 0.5987090304435845
[{'role': 'user', 'content': ' Hola'}, {'role': 'assistant', 'content': "Hola! ¡Bienvenido! How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm here to help with any information you might need."}, {'role': 'user', 'content': ' cuanto es 2 más 2'}]
[{'role': 'user', 'content': ' Hola'}, {'role': 'assistant', 'content': "Hola! ¡Bienvenido! How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm here to help with any information you might need."}, {'role': 'user', 'content': ' cuanto es 2 más 2'}]
 > Text splitted to sentences.
['¡fácil!', '2 + 2 = 4']


...

 > Processing time: 3.787071943283081
 > Real-time factor: 0.5603455574229114
[{'role': 'user', 'content': ' Hola'}, {'role': 'assistant', 'content': "Hola! ¡Bienvenido! How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm here to help with any information you might need."}, {'role': 'user', 'content': ' cuanto es 2 más 2'}, {'role': 'assistant', 'content': '¡fácil!\n\n2 + 2 = 4'}, {'role': 'user', 'content': ' por 5'}]
[{'role': 'user', 'content': ' Hola'}, {'role': 'assistant', 'content': "Hola! ¡Bienvenido! How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm here to help with any information you might need."}, {'role': 'user', 'content': ' cuanto es 2 más 2'}, {'role': 'assistant', 'content': '¡fácil!\n\n2 + 2 = 4'}, {'role': 'user', 'content': ' por 5'}]
 > Text splitted to sentences.
['¡otra suma!', '4 x 5 = 20']


INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 17:58:49] "POST / HTTP/1.1" 200 -


 > Processing time: 2.7698121070861816
 > Real-time factor: 0.4948337192219528
[{'role': 'user', 'content': ' Hola'}, {'role': 'assistant', 'content': "Hola! ¡Bienvenido! How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm here to help with any information you might need."}, {'role': 'user', 'content': ' cuanto es 2 más 2'}, {'role': 'assistant', 'content': '¡fácil!\n\n2 + 2 = 4'}, {'role': 'user', 'content': ' por 5'}, {'role': 'assistant', 'content': '¡otra suma!\n\n4 x 5 = 20'}, {'role': 'user', 'content': ' entre dos'}]
[{'role': 'user', 'content': ' Hola'}, {'role': 'assistant', 'content': "Hola! ¡Bienvenido! How can I assist you today? Do you have a specific question or topic you'd like to discuss? I'm here to help with any information you might need."}, {'role': 'user', 'content': ' cuanto es 2 más 2'}, {'role': 'assistant', 'content': '¡fácil!\n\n2 + 2 = 4'}, {'role': 'user', 'content': ' por 5'}, {'role': 'assistant', 'content': '¡ot

INFO:werkzeug:127.0.0.1 - - [09/Jun/2024 17:59:05] "POST / HTTP/1.1" 200 -


 > Processing time: 2.662397861480713
 > Real-time factor: 0.498385907749675


# Conclusiones

Gracias a los distintos modelos de lenguaje fuimos capaces de analizar la voz, transcribirla, clonarla, generar texto, y generar una respuesta hablada.

# Tu turno

Este método tiene un problema que es algo lento pues debe esperar que un modelo termine para que otro empiece a procesar.

Para solucionarlo existe el `streaming`, el cuál casi todos los modelos lo permiten y que hace que el modelo genere salidas que puedan ser escuchadas por el sistema para hacer algo como imprimir el texto generado en tiempo real, o incluso llamar otros modelos como el de generación de voz mientras se sigue creando la respuesta.

También pueden haber modelos de voz que por streaming generen batches de voz de unos cuantos segundos por ejemplo, mientras se sigue procesando el resto.